# Analyse de données

## Préambule

Le jeu de données pour l'exercice est le suivant : https://www.kaggle.com/etiennelq/french-employment-by-town

Ce jeu de données comprend :

- 5 fichiers CSV
- 2 fichiers GeoJSON

Le descriptif de chaque fichier est inclus dans le lien Kaggle ci-dessus.

## Les 3 fichiers à utiliser dans l'exercice sont :

- base_etablissement_par_tranche_effectif.csv
- population.csv
- name_geographic_information_modified.csv
- net_salary_per_town_categories.csv

## Questions

1. Trouver le top 3 des communes ayant le salaire moyen le plus élevé.
2. Trouver la liste des communes où les femmes gagnent en moyenne plus que les hommes.
3. Trouver le top 3 des communes ayant le plus de personnes de 15-30 ans (inclus).
4. Trouver le top 3 des communes ayant le plus grand rapport de 15-30 ans / population totale de la commune, pour les communes de plus de 100 000 habitants.
5. Trouver le top 3 des communes ayant le salaire moyen le plus élevé, parmi les communes de plus de 100 000 habitants.
6. Trouver le top 3 des communes ayant le salaire moyen le plus bas, parmi les communes de plus de 100 000 habitants.
7. Trouver le top 3 des regions ayant le plus de communes.


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sqlf

spark = SparkSession.builder.getOrCreate()
df_employment = spark.read.csv('../data/employment/net_salary_per_town_categories.csv', header=True, inferSchema=True)

In [75]:
# 1. Trouver le top 3 des communes ayant le salaire moyen le plus élevé.
df_employment.select('CODGEO', 'LIBGEO', 'SNHM14')\
            .withColumnsRenamed({'CODGEO': 'cp_commune',
                                 'LIBGEO': 'nom_commune',
                                 'SNHM14': 'salaire_moyen'})\
            .orderBy(sqlf.col('salaire_moyen').desc())\
            .limit(3)\
            .show()

+----------+--------------------+-------------+
|cp_commune|         nom_commune|salaire_moyen|
+----------+--------------------+-------------+
|     78571|Saint-Nom-la-Bret...|         43.3|
|     78233|        Feucherolles|         38.7|
|     78251|           Fourqueux|         38.6|
+----------+--------------------+-------------+



In [15]:
# 2. Trouver la liste des communes où les femmes gagnent en moyenne plus que les hommes.
df_employment.filter(sqlf.col('SNHMF14') > sqlf.col('SNHMH14'))\
            .select('CODGEO', 'LIBGEO')\
            .withColumnsRenamed({'CODGEO': 'cp_commune',
                                 'LIBGEO': 'nom_commune'})\
            .show()

+----------+--------------------+
|cp_commune|         nom_commune|
+----------+--------------------+
|     97109|           Gourbeyre|
|     97313|Montsinéry-Tonneg...|
+----------+--------------------+



In [ ]:
df_population = spark.read.csv('../data/employment/population.csv', header=True, inferSchema=True)

In [55]:
# 3. Trouver le top 3 des communes ayant le plus de personnes de 15-30 ans (inclus).
df_population.filter(sqlf.col('AGEQ80_17').isin(list(range(15, 30))))\
            .groupby('CODGEO', 'LIBGEO')\
            .agg(sqlf.sum('NB').alias('nb_personnes_15_29_ans'))\
            .orderBy(sqlf.col('nb_personnes_15_29_ans').desc())\
            .limit(3)\
            .show()

df_population.filter(sqlf.col('AGEQ80_17').isin(list(range(15, 34))))\
            .groupby('CODGEO', 'LIBGEO')\
            .agg(sqlf.sum('NB').alias('nb_personnes_15_34_ans'))\
            .orderBy(sqlf.col('nb_personnes_15_34_ans').desc())\
            .limit(3)\
            .show()

+------+---------+----------------------+
|CODGEO|   LIBGEO|nb_personnes_15_29_ans|
+------+---------+----------------------+
| 75056|    Paris|                500790|
| 13055|Marseille|                159319|
| 31555| Toulouse|                143981|
+------+---------+----------------------+

+------+---------+----------------------+
|CODGEO|   LIBGEO|nb_personnes_15_34_ans|
+------+---------+----------------------+
| 75056|    Paris|                685809|
| 13055|Marseille|                214987|
| 31555| Toulouse|                184883|
+------+---------+----------------------+



In [72]:
# 4. Trouver le top 3 des communes ayant le plus grand rapport de 15-30 ans / population totale de la commune, pour les communes de plus de 100 000 habitants.
df_population.groupby('CODGEO', 'LIBGEO')\
            .agg(sqlf.sum('NB').alias('population_totale'))\
            .filter(sqlf.col('population_totale') > 100000)\
            .join(df_population.filter(sqlf.col('AGEQ80_17').isin(list(range(15, 34))))\
                                .groupby('CODGEO', 'LIBGEO')\
                                .agg(sqlf.sum('NB').alias('nb_personnes_15_34_ans')),
                  on=['CODGEO', 'LIBGEO'])\
            .withColumn('ratio_15_34_sur_popul_totale',
                        sqlf.round(sqlf.col('nb_personnes_15_34_ans') / sqlf.col('population_totale'), 4))\
            .orderBy(sqlf.col('ratio_15_34_sur_popul_totale').desc())\
            .limit(3)\
            .show()

+------+--------+-----------------+----------------------+----------------------------+
|CODGEO|  LIBGEO|population_totale|nb_personnes_15_34_ans|ratio_15_34_sur_popul_totale|
+------+--------+-----------------+----------------------+----------------------------+
| 59350|   Lille|           225680|                100536|                      0.4455|
| 31555|Toulouse|           455548|                184883|                      0.4058|
| 35238|  Rennes|           202962|                 81445|                      0.4013|
+------+--------+-----------------+----------------------+----------------------------+



In [76]:
# 5. Trouver le top 3 des communes ayant le salaire moyen le plus élevé, parmi les communes de plus de 100 000 habitants.
df_population.groupby('CODGEO', 'LIBGEO')\
            .agg(sqlf.sum('NB').alias('population_totale'))\
            .filter(sqlf.col('population_totale') > 100000)\
            .join(df_employment.select('CODGEO', 'LIBGEO', 'SNHM14')\
                                .withColumnsRenamed({'SNHM14': 'salaire_moyen'}),
                 on=['CODGEO', 'LIBGEO'])\
            .orderBy(sqlf.col('salaire_moyen').desc())\
            .limit(3)\
            .show()

+------+--------------------+-----------------+-------------+
|CODGEO|              LIBGEO|population_totale|salaire_moyen|
+------+--------------------+-----------------+-------------+
| 92012|Boulogne-Billancourt|           114959|         24.4|
| 75056|               Paris|          2173279|         22.2|
| 13001|     Aix-en-Provence|           134257|         16.1|
+------+--------------------+-----------------+-------------+



In [77]:
# 6. Trouver le top 3 des communes ayant le salaire moyen le plus bas, parmi les communes de plus de 100 000 habitants.
df_population.groupby('CODGEO', 'LIBGEO')\
            .agg(sqlf.sum('NB').alias('population_totale'))\
            .filter(sqlf.col('population_totale') > 100000)\
            .join(df_employment.select('CODGEO', 'LIBGEO', 'SNHM14')\
                                .withColumnsRenamed({'SNHM14': 'salaire_moyen'}),
                 on=['CODGEO', 'LIBGEO'])\
            .orderBy(sqlf.col('salaire_moyen'))\
            .limit(3)\
            .show()

+------+---------+-----------------+-------------+
|CODGEO|   LIBGEO|population_totale|salaire_moyen|
+------+---------+-----------------+-------------+
| 66136|Perpignan|           117199|         11.8|
| 68224| Mulhouse|           107352|         12.1|
| 29019|    Brest|           134216|         12.5|
+------+---------+-----------------+-------------+



In [81]:
# 7. Trouver le top 3 des regions ayant le plus de communes.
spark = SparkSession.builder.getOrCreate()
df_geo = spark.read.csv('../data/employment/name_geographic_information.csv', header=True, inferSchema=True)
df_geo.groupby('code_région', 'nom_région')\
        .agg(sqlf.count('*').alias('nb_de_communes'))\
        .orderBy(sqlf.col('nb_de_communes').desc())\
        .show()

+-----------+--------------------+--------------+
|code_région|          nom_région|nb_de_communes|
+-----------+--------------------+--------------+
|         73|       Midi-Pyrénées|          3028|
|         82|         Rhône-Alpes|          2890|
|         41|            Lorraine|          2336|
|         72|           Aquitaine|          2300|
|         22|            Picardie|          2295|
|         26|           Bourgogne|          2050|
|         21|   Champagne-Ardenne|          1954|
|         24|              Centre|          1849|
|         25|     Basse-Normandie|          1815|
|         43|       Franche-Comté|          1787|
|         31|  Nord-Pas-de-Calais|          1558|
|         91|Languedoc-Roussillon|          1552|
|         52|    Pays de la Loire|          1515|
|         54|    Poitou-Charentes|          1465|
|         23|     Haute-Normandie|          1423|
|         11|       Île-de-France|          1314|
|         83|            Auvergne|          1313|
